In [ ]:
!pip install -U 'scikit-learn<0.24'
!pip install sklearn-crfsuite

# YOU NEED TO RESTART THE RUNTIME!!!

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Run this cell to mount your drive to this notebook in order to read the datasets
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import re
import json
import pandas as pd
import numpy as np
from os.path import join
import warnings
warnings.filterwarnings("ignore")

## Read Dataset

In [ ]:
# Put the folder path where the datasets are located
PATH = "/content/drive/My Drive/dataset "

In [ ]:
# function to read data, return list of tuples each tuple represents a token contains word, pos tag, chunk tag, and ner tag
import csv
def read_data(filename) -> list:
  data = []
  sentences = []
  with open(filename) as load_file:
    reader = csv.reader(load_file,delimiter = " ")   # read
   
    for row in reader:
     
      if(len(tuple(row)) != 0):
       
        if tuple(row) == (' ', 'O', 'O'):
          #print("STOP")
          new_data = (' " ',' " ', 'O', 'O')
          data.append(new_data)
        elif tuple(row) == (' ', 'O', 'I-PER'):
          new_data = (' " ',' " ', 'O', 'I-PER')
          data.append(new_data)

        else:
         
          data.append(tuple(row))
      else:
        sentences.append(data)
        data  =[]
    # add last data because there is no empty line after these data           
    if data:
       sentences.append(data)
    
     
  
    return sentences
  




In [ ]:
#Funciton that read data then get Name Entity tag from, this is work as sent2labels

def read_data_labels(filename) -> list:
  data = []
  sentences = []
  with open(filename) as load_file:
    reader = csv.reader(load_file,delimiter = " ")   # read
   
    for row in reader:
     
      if(len(tuple(row)) != 0):
       
        if tuple(row) == (' ', 'O', 'O'):
          new_data = (' " ',' " ', 'O', 'O')
          data.append(new_data[3])
        elif tuple(row) == (' ', 'O', 'I-PER'):
          new_data = (' " ',' " ', 'O', 'I-PER')
          data.append(new_data[3])
          #print("STOP")
          
        else:
         
          data.append(tuple(row)[3])
      else:
        sentences.append(data)
        data  =[]
    # add last data because there is no empty line after these data           
    if data:
       sentences.append(data)
    
     
  
    return sentences

In [ ]:
# read data with your custom function
train_data = read_data("train.txt")
val_data = read_data("valid.txt")
test_data = read_data("test.txt")

In [ ]:
    '''
      #first_list.append(doc[i][j][3])
      #print("I m at first list")
      first_list.append(doc[i][j][3])
      #print(first_list)
    #print("I am at second list")
  second_list.append(first_list)     
  #second_list.append(first_list) 
  '''  

'\n  #first_list.append(doc[i][j][3])\n  #print("I m at first list")\n  first_list.append(doc[i][j][3])\n  #print(first_list)\n#print("I am at second list")\n  second_list.append(first_list)     \n  #second_list.append(first_list) \n  '

In [ ]:
#Hata bulma fonksiyonu

def entitiytager(doc):
  
  first_list = []
  second_list = []
  counter = 0
  for i in range(len(doc)):
    len1 = len(doc[i])
    for j in range(len1):
      print(doc[i][j][3])
      print(i,j)
     
   


In [ ]:
train_data[14986][3][3]

'O'

# Create Gazetteer

In [ ]:
# load wikipedia pages

import os,json

import glob
contents = []
gazetter= []
path_to_json = "/content/drive/My Drive/wikipedia_pages/"

for file_name in [file for file in os.listdir(path_to_json) if file.endswith('.json')]:
  with open(path_to_json + file_name) as json_file:
    data = json.load(json_file)
    contents.append(data)
    given_text =re.findall('&gt;([a-zA-Z]+)&lt;',data['text']) # find interwiki links from the content of the pages
    for i in range(len(given_text)):
      gazetter.append(given_text[i])
 
    
    # you can also define rules to improve your gazetteer


In [ ]:
# print the size of your gazetteer
len(gazetter)

282331

# Models

## Conditional Random Fields (CRF)

### Extract features for CRF

In [ ]:
import nltk
from nltk.stem import PorterStemmer

nltk.download('stopwords')
from nltk.corpus import stopwords

import sklearn_crfsuite
from sklearn.metrics import make_scorer
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.model_selection import GridSearchCV

from string import punctuation

from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
porter_stemmer = PorterStemmer()
stopwords = stopwords.words("english")

In [ ]:
#Functions to help create features for CRF


def word_shaper(word): #7
  shapecreater = " "
  shortshapecreater = " "
  for i in range(len(word)):
    if word[i].isupper():
     shapecreater = shapecreater+'X'
    elif word[i].islower():
      shapecreater = shapecreater + 'x'
    elif word[i].isdigit():
      shapecreater = shapecreater+ 'd'
    elif word[i] in punctuation:
      shapecreater = shapecreater+ word[i]
  return shapecreater

def short_shaper(word):
  shortshapecreater = " "
  if len(word) == 0:
    return shortshapecreater
  else:
    if len(word) >= 3:
      if word[0].isupper() and word[1] == "’" and word[2].isupper():
        shortshapecreater =  "X'Xx"
      elif word[0].isupper() == False and word[0] not in punctuation:
        shortshapecreater = 'x'
      elif word[0].isupper() and word[1] != "'":
        shortshapecreater = 'Xx' 
  for i in range(len(word)):
     
  
     if word[i] in punctuation:
      shortshapecreater = shortshapecreater+word[i]

  return shortshapecreater

 

def number_checker(word):  #9
  for i in range(len(word)):
    if word[i].isdigit():
      return True
    else:
      return False

def hypen_checker(word):  #10
  
  for i in range(len(word)):
    if word[i] == "-":
      return True
    else:
      return False


def upper_digit_dash(word): #11
  upper = False
  digit = False
  dash = False
  if word.isupper():
    upper = True
  else:
    return False
  for i in range(len(word)):

    if word[i].isdigit():
      digit = True
    elif word[i] == '-':
      dash = True
  return (upper and digit and dash)


def prefix(word): #12
  pre1 = " "
  pre2 = " "
  pre3 = " "
  pre4 = " "

  for i in range(len(word)):
    if i == 0:
      pre1 = pre1+word[i]
    if i == 1:
      pre2 = pre1+word[i]
    if i == 2:
      pre3 =pre2+word[i]
    if i ==3:
      pre4 = pre3+word[i]
 
  return pre1,pre2,pre3,pre4

def suffix(word): #13
  suf1 = " "
  suf2 = " "
  suf3 = " "
  suf4 = " "

  for i in range(len(word)):
    if i ==0:
      suf1 = suf1+(word[len(word)-1-i])
    if i == 1:
      suf2 = suf1+(word[len(word)-1-i])
    if i == 2:
      suf3 = suf2+(word[len(word)-1-i])
    if i == 3:
      suf4 = suf3+(word[len(word)-1-i])
  return suf1,suf2,suf3,suf4

def allupper(word): #14
  for i in range(len(word)):
    if word[i].isupper() == False:
      return False
    
  return True

def stopwordchecker(word): #15
  if word in stopwords:
    return True
  else:
   return False

def isin_gazetter(word):  #19  #Güncelleme yap

  if word in gazetter:
   return True         #use dictionary
  else:
    return False  



In [ ]:

def word2features(doc, i):
  
  word = doc[i][0]
  postag = doc[i][1]
  chunktag = doc[i][2]
  #name_entitiy = doc[i][3]
  before = ""
  after = ""
  short_shape = short_shaper(word)
  before_short_shape = " "
  after_short_shape = " "
 
 
  if i>0:                         #16 Neighbors

    beforeword = doc[i-1][0]
    before = beforeword
    before_short_shape = short_shaper(before)
   
    
  if i != len(doc)-1:
    nextword  =doc[i+1][0]
    after =nextword
    after_short_shape = short_shaper(after)
   
    
 
  


  stem_word = porter_stemmer.stem(word)

  if word == "-DOCSTART-":
    start_sentence = True
  else:
    start_sentence = False

  if word == '.':
    end_sentence = True
  else: 
    end_sentence = False


  if word[0].isupper():
    is_upper = True
  else:
    is_upper = False
  
  word_shape = word_shaper(word)
  next_word_shape = word_shaper(after)        #18
  before_word_shape = word_shaper(before)     #18
  
  
  contain_num = number_checker(word)
  is_hypen = hypen_checker(word)
  is_up_digit_dash = upper_digit_dash(word)
  prefixes = prefix(word)
  sufixes = suffix(word)
  is_all_upper = allupper(word)
  is_stopword = stopwordchecker(word)
  is_in_gazeeter = isin_gazetter(word)
  
  
 
  
  features = { 
  
  'stem': stem_word,
  'pos_tag': postag,
   'chunk tag': chunktag,
   'start_of_sentence': start_sentence,
   'end_of_sentence': end_sentence,
   #'name_entity':name_entitiy,
   'Is_uppercase_letter': is_upper,
   'Word_Shape': word_shape,
   'Short_shape':short_shape,
   'Contain_Number': contain_num,
   'Contain_Hypen': is_hypen,
   'Contain upper, digit and dash': is_up_digit_dash,
   'prefix 1':prefixes[0],
   'prefix 2':prefixes[1],
   'prefix 3':prefixes[2],
   'prefix 4':prefixes[3],
   'sufix 1': sufixes[0],
   'sufix 2': sufixes[1],
   'sufix 3': sufixes[2],
   'sufix 4': sufixes[3],
   'All_upper': is_all_upper,
   'Is_stopword':is_stopword,
   'Before neighbor words':before,
   'After neighbor word':after,
   'Before word shape':before_word_shape,
   'After word shape':next_word_shape,
   'Before short shape':before_short_shape,
   'After short shape':after_short_shape,
   'Presence of in gazetter':is_in_gazeeter

  
      
  }

  return features




# define function to process each token given a sentence
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]
 


# get named entity labels from the sentence
def sent2labels(doc,i) :                 #Buraya bak
  name_tag = doc[i][3]
  return name_tag


def label_for_sent(sent):
  return [sent2labels(sent,i) for i in range(len(sent))]

Create functions for each feature to be added

In [ ]:
def feature1(doc,i):
  word = doc[i][0]
  stem_word = porter_stemmer.stem(word)

  features = { 
  
  'stem': stem_word
  }

  return features


In [ ]:
def feature2(doc,i):

  word = doc[i][0]
  stem_word = porter_stemmer.stem(word)
  postag = doc[i][1]


  features = { 
  
  'stem': stem_word,
  'pos_tag': postag,
  }
  return features

In [ ]:
def feature3(doc,i):

  word = doc[i][0]
  stem_word = porter_stemmer.stem(word)
  postag = doc[i][1]
  chunktag = doc[i][2]


  features = { 
  
  'stem': stem_word,
  'pos_tag': postag,
  'chunk tag': chunktag
  }
  return features

In [ ]:
def feature4(doc,i):
  word = doc[i][0]
  postag = doc[i][1]
  chunktag = doc[i][2]
  
  stem_word = porter_stemmer.stem(word)

  if word == "-DOCSTART-":
    start_sentence = True
  else:
    start_sentence = False

  if word == '.':
    end_sentence = True
  else: 
    end_sentence = False

  features = { 
  
  'stem': stem_word,
  'pos_tag': postag,

   'chunk tag': chunktag,
   'start_of_sentence': start_sentence,
   
  }
  return features

In [ ]:
def feature5(doc,i):
  word = doc[i][0]
  postag = doc[i][1]
  chunktag = doc[i][2]
  
  stem_word = porter_stemmer.stem(word)

  if word == "-DOCSTART-":
    start_sentence = True
  else:
    start_sentence = False

  if word == '.':
    end_sentence = True
  else: 
    end_sentence = False

  features = { 
  
  'stem': stem_word,
  'pos_tag': postag,

   'chunk tag': chunktag,
   'start_of_sentence': start_sentence,
   'end_of_sentence': end_sentence,
  }
  return features

In [ ]:
def feature6(doc,i):
  word = doc[i][0]
  postag = doc[i][1]
  chunktag = doc[i][2]

  stem_word = porter_stemmer.stem(word)

  if word == "-DOCSTART-":
    start_sentence = True
  else:
    start_sentence = False

  if word == '.':
    end_sentence = True
  else: 
    end_sentence = False


  if word[0].isupper():
    is_upper = True
  else:
    is_upper = False 
   
   
  features = { 
  
  'stem': stem_word,
  'pos_tag': postag,
   'chunk tag': chunktag,
   'start_of_sentence': start_sentence,
   'end_of_sentence': end_sentence,
   'Is_uppercase_letter': is_upper,
   }
  return features

In [ ]:
def feature7(doc, i):
  
  word = doc[i][0]
  postag = doc[i][1]
  chunktag = doc[i][2]
  #name_entitiy = doc[i][3]
  before = ""
  after = ""
  if i>0:                         #16 Neighbors

    beforeword = doc[i-1][0]
    before = beforeword
  if i != len(doc)-1:
    nextword  =doc[i+1][0]
    after =nextword
 
  


  stem_word = porter_stemmer.stem(word)

  if word == "-DOCSTART-":
    start_sentence = True
  else:
    start_sentence = False

  if word == '.':
    end_sentence = True
  else: 
    end_sentence = False


  if word[0].isupper():
    is_upper = True
  else:
    is_upper = False
  
  word_shape = word_shaper(word)
 
  
 
  
  features = { 
  
  'stem': stem_word,
  'pos_tag': postag,
   'chunk tag': chunktag,
   'start_of_sentence': start_sentence,
   'end_of_sentence': end_sentence,
   #'name_entity':name_entitiy,
   'Is_uppercase_letter': is_upper,
   'Word_Shape': word_shape,
   

    #8 add short shape
    #17 add short shape of neighbors
      
  }

  return features

In [ ]:
def feature8(doc, i): #Short shape
  
  word = doc[i][0]
  postag = doc[i][1]
  chunktag = doc[i][2]
  #name_entitiy = doc[i][3]
  before = ""
  after = ""
  short_shape = short_shaper(word)
  if i>0:                         #16 Neighbors

    beforeword = doc[i-1][0]
    before = beforeword
  if i != len(doc)-1:
    nextword  =doc[i+1][0]
    after =nextword
 
  


  stem_word = porter_stemmer.stem(word)

  if word == "-DOCSTART-":
    start_sentence = True
  else:
    start_sentence = False

  if word == '.':
    end_sentence = True
  else: 
    end_sentence = False


  if word[0].isupper():
    is_upper = True
  else:
    is_upper = False
  
  word_shape = word_shaper(word)
 
  
 
  
  features = { 
  
  'stem': stem_word,
  'pos_tag': postag,
   'chunk tag': chunktag,
   'start_of_sentence': start_sentence,
   'end_of_sentence': end_sentence,
   #'name_entity':name_entitiy,
   'Is_uppercase_letter': is_upper,
   'Word_Shape': word_shape,
   'Short_shape':short_shape,
   
      
  }

  return features

In [ ]:
def feature9(doc, i):
  
  word = doc[i][0]
  postag = doc[i][1]
  chunktag = doc[i][2]
  #name_entitiy = doc[i][3]
  before = ""
  after = ""
  short_shape = short_shaper(word)
  if i>0:                         #16 Neighbors

    beforeword = doc[i-1][0]
    before = beforeword
  if i != len(doc)-1:
    nextword  =doc[i+1][0]
    after =nextword
 
  


  stem_word = porter_stemmer.stem(word)

  if word == "-DOCSTART-":
    start_sentence = True
  else:
    start_sentence = False

  if word == '.':
    end_sentence = True
  else: 
    end_sentence = False


  if word[0].isupper():
    is_upper = True
  else:
    is_upper = False
  
  word_shape = word_shaper(word)
  next_word_shape = word_shaper(after)        #18
  before_word_shape = word_shaper(before)     #18
  contain_num = number_checker(word)
  is_hypen = hypen_checker(word)
  is_up_digit_dash = upper_digit_dash(word)
  prefixes = prefix(word)
  sufixes = suffix(word)
  is_all_upper = allupper(word)
  is_stopword = stopwordchecker(word)
  
  
 
  
  features = { 
  
  'stem': stem_word,
  'pos_tag': postag,
   'chunk tag': chunktag,
   'start_of_sentence': start_sentence,
   'end_of_sentence': end_sentence,
   #'name_entity':name_entitiy,
   'Is_uppercase_letter': is_upper,
   'Word_Shape': word_shape,
   'Short_shape':short_shape,
   'Contain_Number': contain_num,
  
      
  }

  return features

In [ ]:
def feature10(doc, i):
  
  word = doc[i][0]
  postag = doc[i][1]
  chunktag = doc[i][2]
  #name_entitiy = doc[i][3]
  before = ""
  after = ""
  short_shape = short_shaper(word)
  if i>0:                         #16 Neighbors

    beforeword = doc[i-1][0]
    before = beforeword
  if i != len(doc)-1:
    nextword  =doc[i+1][0]
    after =nextword
 
  


  stem_word = porter_stemmer.stem(word)

  if word == "-DOCSTART-":
    start_sentence = True
  else:
    start_sentence = False

  if word == '.':
    end_sentence = True
  else: 
    end_sentence = False


  if word[0].isupper():
    is_upper = True
  else:
    is_upper = False
  
  word_shape = word_shaper(word)
  next_word_shape = word_shaper(after)        #18
  before_word_shape = word_shaper(before)     #18
  contain_num = number_checker(word)
  is_hypen = hypen_checker(word)
  is_up_digit_dash = upper_digit_dash(word)
  prefixes = prefix(word)
  sufixes = suffix(word)
  is_all_upper = allupper(word)
  is_stopword = stopwordchecker(word)
  
  
 
  
  features = { 
  
  'stem': stem_word,
  'pos_tag': postag,
   'chunk tag': chunktag,
   'start_of_sentence': start_sentence,
   'end_of_sentence': end_sentence,
   #'name_entity':name_entitiy,
   'Is_uppercase_letter': is_upper,
   'Word_Shape': word_shape,
   'Short_shape':short_shape,
   'Contain_Number': contain_num,
   'Contain_Hypen': is_hypen,

      
  }

  return features

In [ ]:
def feature11(doc, i):
  
  word = doc[i][0]
  postag = doc[i][1]
  chunktag = doc[i][2]
  #name_entitiy = doc[i][3]
  before = ""
  after = ""
  short_shape = short_shaper(word)
  if i>0:                         #16 Neighbors

    beforeword = doc[i-1][0]
    before = beforeword
  if i != len(doc)-1:
    nextword  =doc[i+1][0]
    after =nextword
 
  


  stem_word = porter_stemmer.stem(word)

  if word == "-DOCSTART-":
    start_sentence = True
  else:
    start_sentence = False

  if word == '.':
    end_sentence = True
  else: 
    end_sentence = False


  if word[0].isupper():
    is_upper = True
  else:
    is_upper = False
  
  word_shape = word_shaper(word)
  next_word_shape = word_shaper(after)        #18
  before_word_shape = word_shaper(before)     #18
  contain_num = number_checker(word)
  is_hypen = hypen_checker(word)
  is_up_digit_dash = upper_digit_dash(word)
  prefixes = prefix(word)
  sufixes = suffix(word)
  is_all_upper = allupper(word)
  is_stopword = stopwordchecker(word)
 
  
 
  
  features = { 
  
  'stem': stem_word,
  'pos_tag': postag,
   'chunk tag': chunktag,
   'start_of_sentence': start_sentence,
   'end_of_sentence': end_sentence,
   #'name_entity':name_entitiy,
   'Is_uppercase_letter': is_upper,
   'Word_Shape': word_shape,
   'Short_shape':short_shape,
   'Contain_Number': contain_num,
   'Contain_Hypen': is_hypen,
   'Contain upper, digit and dash': is_up_digit_dash,
  
      
  }

  return features

In [ ]:
def feature12(doc, i): #prefix
  
  word = doc[i][0]
  postag = doc[i][1]
  chunktag = doc[i][2]
  #name_entitiy = doc[i][3]
  before = ""
  after = ""
  short_shape = short_shaper(word)
  if i>0:                         #16 Neighbors

    beforeword = doc[i-1][0]
    before = beforeword
  if i != len(doc)-1:
    nextword  =doc[i+1][0]
    after =nextword
 
  


  stem_word = porter_stemmer.stem(word)

  if word == "-DOCSTART-":
    start_sentence = True
  else:
    start_sentence = False

  if word == '.':
    end_sentence = True
  else: 
    end_sentence = False


  if word[0].isupper():
    is_upper = True
  else:
    is_upper = False
  
  word_shape = word_shaper(word)
  next_word_shape = word_shaper(after)        #18
  before_word_shape = word_shaper(before)     #18
  contain_num = number_checker(word)
  is_hypen = hypen_checker(word)
  is_up_digit_dash = upper_digit_dash(word)
  prefixes = prefix(word)
  
  
 
  
  features = { 
  
  'stem': stem_word,
  'pos_tag': postag,
   'chunk tag': chunktag,
   'start_of_sentence': start_sentence,
   'end_of_sentence': end_sentence,
   #'name_entity':name_entitiy,
   'Is_uppercase_letter': is_upper,
   'Word_Shape': word_shape,
   'Short_shape':short_shape,
   'Contain_Number': contain_num,
   'Contain_Hypen': is_hypen,
   'Contain upper, digit and dash': is_up_digit_dash,
   'prefix 1':prefixes[0],
   'prefix 2':prefixes[1],
   'prefix 3':prefixes[2],
   'prefix 4':prefixes[3],
   

 
      
  }

  return features

In [ ]:
def feature13(doc, i): #sufixes
  
  word = doc[i][0]
  postag = doc[i][1]
  chunktag = doc[i][2]
  #name_entitiy = doc[i][3]
  before = ""
  after = ""
  short_shape = short_shaper(word)
  if i>0:                         #16 Neighbors

    beforeword = doc[i-1][0]
    before = beforeword
  if i != len(doc)-1:
    nextword  =doc[i+1][0]
    after =nextword
 
  


  stem_word = porter_stemmer.stem(word)

  if word == "-DOCSTART-":
    start_sentence = True
  else:
    start_sentence = False

  if word == '.':
    end_sentence = True
  else: 
    end_sentence = False


  if word[0].isupper():
    is_upper = True
  else:
    is_upper = False
  
  word_shape = word_shaper(word)
  next_word_shape = word_shaper(after)        #18
  before_word_shape = word_shaper(before)     #18
  contain_num = number_checker(word)
  is_hypen = hypen_checker(word)
  is_up_digit_dash = upper_digit_dash(word)
  prefixes = prefix(word)
  sufixes = suffix(word)
  
  features = { 
  
  'stem': stem_word,
  'pos_tag': postag,
   'chunk tag': chunktag,
   'start_of_sentence': start_sentence,
   'end_of_sentence': end_sentence,
   #'name_entity':name_entitiy,
   'Is_uppercase_letter': is_upper,
   'Word_Shape': word_shape,
   'Short_shape':short_shape,
   'Contain_Number': contain_num,
   'Contain_Hypen': is_hypen,
   'Contain upper, digit and dash': is_up_digit_dash,
   'prefix 1':prefixes[0],
   'prefix 2':prefixes[1],
   'prefix 3':prefixes[2],
   'prefix 4':prefixes[3],
   'sufix 1': sufixes[0],
   'sufix 2': sufixes[1],
   'sufix 3': sufixes[2],
   'sufix 4': sufixes[3],
   

    #8 add short shape
    #17 add short shape of neighbors
      
  }

  return features

In [ ]:
def feature14(doc, i):
  
  word = doc[i][0]
  postag = doc[i][1]
  chunktag = doc[i][2]
  #name_entitiy = doc[i][3]
  before = ""
  after = ""
  short_shape = short_shaper(word)
  if i>0:                         #16 Neighbors

    beforeword = doc[i-1][0]
    before = beforeword
  if i != len(doc)-1:
    nextword  =doc[i+1][0]
    after =nextword
 
  


  stem_word = porter_stemmer.stem(word)

  if word == "-DOCSTART-":
    start_sentence = True
  else:
    start_sentence = False

  if word == '.':
    end_sentence = True
  else: 
    end_sentence = False


  if word[0].isupper():
    is_upper = True
  else:
    is_upper = False
  
  word_shape = word_shaper(word)
  next_word_shape = word_shaper(after)        #18
  before_word_shape = word_shaper(before)     #18
  contain_num = number_checker(word)
  is_hypen = hypen_checker(word)
  is_up_digit_dash = upper_digit_dash(word)
  prefixes = prefix(word)
  sufixes = suffix(word)
  is_all_upper = allupper(word)
  is_stopword = stopwordchecker(word)
  
  
 
  
  features = { 
  
  'stem': stem_word,
  'pos_tag': postag,
   'chunk tag': chunktag,
   'start_of_sentence': start_sentence,
   'end_of_sentence': end_sentence,
   #'name_entity':name_entitiy,
   'Is_uppercase_letter': is_upper,
   'Word_Shape': word_shape,
   'Short_shape':short_shape,
   'Contain_Number': contain_num,
   'Contain_Hypen': is_hypen,
   'Contain upper, digit and dash': is_up_digit_dash,
   'prefix 1':prefixes[0],
   'prefix 2':prefixes[1],
   'prefix 3':prefixes[2],
   'prefix 4':prefixes[3],
   'sufix 1': sufixes[0],
   'sufix 2': sufixes[1],
   'sufix 3': sufixes[2],
   'sufix 4': sufixes[3],
   'All_upper': is_all_upper,
   

    #8 add short shape
    #17 add short shape of neighbors
      
  }

  return features

In [ ]:
def feature15(doc, i):
  
  word = doc[i][0]
  postag = doc[i][1]
  chunktag = doc[i][2]
  #name_entitiy = doc[i][3]
  before = ""
  after = ""
  short_shape = short_shaper(word)
  if i>0:                         #16 Neighbors

    beforeword = doc[i-1][0]
    before = beforeword
  if i != len(doc)-1:
    nextword  =doc[i+1][0]
    after =nextword
 
  


  stem_word = porter_stemmer.stem(word)

  if word == "-DOCSTART-":
    start_sentence = True
  else:
    start_sentence = False

  if word == '.':
    end_sentence = True
  else: 
    end_sentence = False


  if word[0].isupper():
    is_upper = True
  else:
    is_upper = False
  
  word_shape = word_shaper(word)
  next_word_shape = word_shaper(after)        #18
  before_word_shape = word_shaper(before)     #18
  contain_num = number_checker(word)
  is_hypen = hypen_checker(word)
  is_up_digit_dash = upper_digit_dash(word)
  prefixes = prefix(word)
  sufixes = suffix(word)
  is_all_upper = allupper(word)
  is_stopword = stopwordchecker(word)
  
  
 
  
  features = { 
  
  'stem': stem_word,
  'pos_tag': postag,
   'chunk tag': chunktag,
   'start_of_sentence': start_sentence,
   'end_of_sentence': end_sentence,
   #'name_entity':name_entitiy,
   'Is_uppercase_letter': is_upper,
   'Word_Shape': word_shape,
   'Short_shape':short_shape,
   'Contain_Number': contain_num,
   'Contain_Hypen': is_hypen,
   'Contain upper, digit and dash': is_up_digit_dash,
   'prefix 1':prefixes[0],
   'prefix 2':prefixes[1],
   'prefix 3':prefixes[2],
   'prefix 4':prefixes[3],
   'sufix 1': sufixes[0],
   'sufix 2': sufixes[1],
   'sufix 3': sufixes[2],
   'sufix 4': sufixes[3],
   'All_upper': is_all_upper,
   'Is_stopword':is_stopword,
  

    #8 add short shape
    #17 add short shape of neighbors
      
  }

  return features

In [ ]:
def feature16(doc, i):
  
  word = doc[i][0]
  postag = doc[i][1]
  chunktag = doc[i][2]
  #name_entitiy = doc[i][3]
  before = ""
  after = ""
  short_shape = short_shaper(word)
  if i>0:                         #16 Neighbors

    beforeword = doc[i-1][0]
    before = beforeword
  if i != len(doc)-1:
    nextword  =doc[i+1][0]
    after =nextword
 
  


  stem_word = porter_stemmer.stem(word)

  if word == "-DOCSTART-":
    start_sentence = True
  else:
    start_sentence = False

  if word == '.':
    end_sentence = True
  else: 
    end_sentence = False


  if word[0].isupper():
    is_upper = True
  else:
    is_upper = False
  
  word_shape = word_shaper(word)
  next_word_shape = word_shaper(after)        #18
  before_word_shape = word_shaper(before)     #18
  contain_num = number_checker(word)
  is_hypen = hypen_checker(word)
  is_up_digit_dash = upper_digit_dash(word)
  prefixes = prefix(word)
  sufixes = suffix(word)
  is_all_upper = allupper(word)
  is_stopword = stopwordchecker(word)
  
  
 
  
  features = { 
  
  'stem': stem_word,
  'pos_tag': postag,
   'chunk tag': chunktag,
   'start_of_sentence': start_sentence,
   'end_of_sentence': end_sentence,
   #'name_entity':name_entitiy,
   'Is_uppercase_letter': is_upper,
   'Word_Shape': word_shape,
   'Contain_Number': contain_num,
   'Short_shape':short_shape,
   'Contain_Hypen': is_hypen,
   'Contain upper, digit and dash': is_up_digit_dash,
   'prefix 1':prefixes[0],
   'prefix 2':prefixes[1],
   'prefix 3':prefixes[2],
   'prefix 4':prefixes[3],
   'sufix 1': sufixes[0],
   'sufix 2': sufixes[1],
   'sufix 3': sufixes[2],
   'sufix 4': sufixes[3],
   'All_upper': is_all_upper,
   'Is_stopword':is_stopword,
   'Before neighbor words':before,
   'After neighbor word':after,
   

    #8 add short shape
    #17 add short shape of neighbors
      
  }

  return features

In [ ]:
def feature17(doc, i):
  
  word = doc[i][0]
  postag = doc[i][1]
  chunktag = doc[i][2]
  #name_entitiy = doc[i][3]
  before = ""
  after = ""
  short_shape = short_shaper(word)
  short_shape_before = ""
  short_shape_after = ""
  if i>0:                         #16 Neighbors

    beforeword = doc[i-1][0]
    before = beforeword
    short_shape_before = short_shaper(before)
  if i != len(doc)-1:
    nextword  =doc[i+1][0]
    after =nextword
    short_shape_after = short_shaper(after)
 
  


  stem_word = porter_stemmer.stem(word)

  if word == "-DOCSTART-":
    start_sentence = True
  else:
    start_sentence = False

  if word == '.':
    end_sentence = True
  else: 
    end_sentence = False


  if word[0].isupper():
    is_upper = True
  else:
    is_upper = False
  
  word_shape = word_shaper(word)
  next_word_shape = word_shaper(after)        #18
  before_word_shape = word_shaper(before)     #18
  contain_num = number_checker(word)
  is_hypen = hypen_checker(word)
  is_up_digit_dash = upper_digit_dash(word)
  prefixes = prefix(word)
  sufixes = suffix(word)
  is_all_upper = allupper(word)
  is_stopword = stopwordchecker(word)
  
  
 
  
  features = { 
  
  'stem': stem_word,
  'pos_tag': postag,
   'chunk tag': chunktag,
   'start_of_sentence': start_sentence,
   'end_of_sentence': end_sentence,
   #'name_entity':name_entitiy,
   'Is_uppercase_letter': is_upper,
   'Word_Shape': word_shape,
   'Contain_Number': contain_num,
   'Short_shape':short_shape,
   'Contain_Hypen': is_hypen,
   'Contain upper, digit and dash': is_up_digit_dash,
   'prefix 1':prefixes[0],
   'prefix 2':prefixes[1],
   'prefix 3':prefixes[2],
   'prefix 4':prefixes[3],
   'sufix 1': sufixes[0],
   'sufix 2': sufixes[1],
   'sufix 3': sufixes[2],
   'sufix 4': sufixes[3],
   'All_upper': is_all_upper,
   'Is_stopword':is_stopword,
   'Before neighbor words':before,
   'After neighbor word':after,
   'Short shape before':short_shape_before,
   "Short shape after": short_shape_after,
   

    #8 add short shape
    #17 add short shape of neighbors
      
  }

  
  return features

In [ ]:
def feature18(doc, i):
  
  word = doc[i][0]
  postag = doc[i][1]
  chunktag = doc[i][2]
  
  before = ""
  after = ""
  short_shape_before = ""
  short_shape_after = ""
  short_shape = short_shaper(word)
  if i>0:                         #16 Neighbors

    beforeword = doc[i-1][0]
    before = beforeword
    short_shape_before = short_shaper(before)
    
  
  if i != len(doc)-1:
    nextword  =doc[i+1][0]
    after =nextword
    short_shape_after = short_shaper(after)
    
 
  


  stem_word = porter_stemmer.stem(word)

  if word == "-DOCSTART-":
    start_sentence = True
  else:
    start_sentence = False

  if word == '.':
    end_sentence = True
  else: 
    end_sentence = False


  if word[0].isupper():
    is_upper = True
  else:
    is_upper = False
  
  word_shape = word_shaper(word)
  next_word_shape = word_shaper(after)        #18
  before_word_shape = word_shaper(before)     #18
  contain_num = number_checker(word)
  is_hypen = hypen_checker(word)
  is_up_digit_dash = upper_digit_dash(word)
  prefixes = prefix(word)
  sufixes = suffix(word)
  is_all_upper = allupper(word)
  is_stopword = stopwordchecker(word)
  
  
 
  
  features = { 
  
  'stem': stem_word,
  'pos_tag': postag,
   'chunk tag': chunktag,
   'start_of_sentence': start_sentence,
   'end_of_sentence': end_sentence,
   #'name_entity':name_entitiy,
   'Is_uppercase_letter': is_upper,
   'Word_Shape': word_shape,
   'Short_shape':short_shape,
   'Contain_Number': contain_num,
   'Contain_Hypen': is_hypen,
   'Contain upper, digit and dash': is_up_digit_dash,
   'prefix 1':prefixes[0],
   'prefix 2':prefixes[1],
   'prefix 3':prefixes[2],
   'prefix 4':prefixes[3],
   'sufix 1': sufixes[0],
   'sufix 2': sufixes[1],
   'sufix 3': sufixes[2],
   'sufix 4': sufixes[3],
   'All_upper': is_all_upper,
   'Is_stopword':is_stopword,
   'Before neighbor words':before,
   'After neighbor word':after,
   'Before word shape':before_word_shape,
   'After word shape':next_word_shape,
   'Short shape before':short_shape_before,
   'Short shape after': short_shape_after,

    #8 add short shape
    #17 add short shape of neighbors
      
  }

  return features



In [ ]:
# prepare inputs and labels
#train_sents = [sent2features(s) for s in train_data]
#val_sents = [sent2features(s) for s in val_data]
#test_sents = [sent2features(s) for s in test_data]

#train_labels = [sent2labels(s) for s in train_data]
#val_labels = [sent2labels(s) for s in val_data]
#test_labels = [sent2labels(s) for s in test_data]

In [ ]:
#Prepare inputs
train_sents = [sent2features(s) for s in train_data]
val_sents = [sent2features(s) for s in val_data]
test_sents = [sent2features(s) for s in test_data]

In [ ]:
#Prepare labels
train_labels = read_data_labels("train.txt")
val_labels = read_data_labels("valid.txt")
test_labels = read_data_labels("test.txt")


In [ ]:
# Set the hyperparameter space that will be scanned.
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)




# fitting the model for grid search 
crf.fit(train_sents, train_labels)

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [ ]:
type(train_sents)

list

In [ ]:
labels = list(crf.classes_)

F1 , Accuracy and Precision score for the All CRF Features implemented 

In [ ]:
# print best parameter after tuning 
y_pred = crf.predict(val_sents)
metrics.flat_f1_score(val_labels, y_pred,
                      average='weighted', labels=labels)
  

#metrics.flat_accuracy_score(val_labels, y_pred)

0.978895261060282

In [ ]:
#Accuracy
metrics.flat_accuracy_score(val_labels, y_pred)

0.9791190042266082

In [ ]:
#Precision
metrics.flat_precision_score(val_labels,y_pred,average = "weighted")

0.9788085755191352

In [ ]:
# calculate f1-score and classification report for test using sklearn_crfsuite.metrics class
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    val_labels, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

           O      0.993     0.997     0.995     42975
       B-LOC      0.917     0.929     0.923      1837
       I-LOC      0.877     0.860     0.868       257
      B-MISC      0.919     0.871     0.894       922
      I-MISC      0.871     0.743     0.802       346
       B-ORG      0.882     0.834     0.857      1341
       I-ORG      0.826     0.830     0.828       751
       B-PER      0.915     0.913     0.914      1842
       I-PER      0.955     0.966     0.960      1307

    accuracy                          0.979     51578
   macro avg      0.906     0.882     0.894     51578
weighted avg      0.979     0.979     0.979     51578



In [ ]:
sorted_labels[0]

'O'

In [ ]:
def train_creater1(train_sentences):
  for key in train_sentences:
    print(key,train_sentences[key])
    

In [ ]:
# start from the stem of the token and add features one by one and train a new model with each feature that you add
def sent2features(sent):
    return [feature1(sent, i) for i in range(len(sent))]

train_sents1 = [sent2features(s) for s in train_data]
val_sents1 = [sent2features(s) for s in val_data]
test_sents1 = [sent2features(s) for s in test_data]

def sent2features(sent):
    return [feature2(sent, i) for i in range(len(sent))]

train_sents2 = [sent2features(s) for s in train_data]
val_sents2 = [sent2features(s) for s in val_data]
test_sents2 = [sent2features(s) for s in test_data]


def sent2features(sent):
    return [feature3(sent, i) for i in range(len(sent))]

train_sents3 = [sent2features(s) for s in train_data]
val_sents3 = [sent2features(s) for s in val_data]
test_sents3 = [sent2features(s) for s in test_data]




def sent2features(sent):
    return [feature4(sent, i) for i in range(len(sent))]

train_sents4 = [sent2features(s) for s in train_data]
val_sents4 = [sent2features(s) for s in val_data]
test_sents4 = [sent2features(s) for s in test_data]


def sent2features(sent):
    return [feature5(sent, i) for i in range(len(sent))]

train_sents5 = [sent2features(s) for s in train_data]
val_sents5 = [sent2features(s) for s in val_data]
test_sents5 = [sent2features(s) for s in test_data]



def sent2features(sent):
    return [feature6(sent, i) for i in range(len(sent))]

train_sents6 = [sent2features(s) for s in train_data]
val_sents6 = [sent2features(s) for s in val_data]
test_sents6 = [sent2features(s) for s in test_data]


def sent2features(sent):
    return [feature7(sent, i) for i in range(len(sent))]

train_sents7 = [sent2features(s) for s in train_data]
val_sents7 = [sent2features(s) for s in val_data]
test_sents7 = [sent2features(s) for s in test_data]



def sent2features(sent):
    return [feature8(sent, i) for i in range(len(sent))]

train_sents8 = [sent2features(s) for s in train_data]
val_sents8 = [sent2features(s) for s in val_data]
test_sents8 = [sent2features(s) for s in test_data]


def sent2features(sent):
    return [feature9(sent, i) for i in range(len(sent))]

train_sents9 = [sent2features(s) for s in train_data]
val_sents9 = [sent2features(s) for s in val_data]
test_sents9 = [sent2features(s) for s in test_data]


def sent2features(sent):
    return [feature10(sent, i) for i in range(len(sent))]

train_sents10 = [sent2features(s) for s in train_data]
val_sents10 = [sent2features(s) for s in val_data]
test_sents10 = [sent2features(s) for s in test_data]

def sent2features(sent):
    return [feature11(sent, i) for i in range(len(sent))]

train_sents11 = [sent2features(s) for s in train_data]
val_sents11 = [sent2features(s) for s in val_data]
test_sents11 = [sent2features(s) for s in test_data]


def sent2features(sent):
    return [feature12(sent, i) for i in range(len(sent))]

train_sents12 = [sent2features(s) for s in train_data]
val_sents12 = [sent2features(s) for s in val_data]
test_sents12 = [sent2features(s) for s in test_data]

def sent2features(sent):
    return [feature13(sent, i) for i in range(len(sent))]

train_sents13 = [sent2features(s) for s in train_data]
val_sents13 = [sent2features(s) for s in val_data]
test_sents13 = [sent2features(s) for s in test_data]


def sent2features(sent):
    return [feature14(sent, i) for i in range(len(sent))]

train_sents14 = [sent2features(s) for s in train_data]
val_sents14 = [sent2features(s) for s in val_data]
test_sents14 = [sent2features(s) for s in test_data]


def sent2features(sent):
    return [feature15(sent, i) for i in range(len(sent))]

train_sents15 = [sent2features(s) for s in train_data]
val_sents15 = [sent2features(s) for s in val_data]
test_sents15 = [sent2features(s) for s in test_data]


def sent2features(sent):
    return [feature16(sent, i) for i in range(len(sent))]

train_sents16 = [sent2features(s) for s in train_data]
val_sents16 = [sent2features(s) for s in val_data]
test_sents16 = [sent2features(s) for s in test_data]


def sent2features(sent):
    return [feature17(sent, i) for i in range(len(sent))]

train_sents17 = [sent2features(s) for s in train_data]
val_sents17 = [sent2features(s) for s in val_data]
test_sents17 = [sent2features(s) for s in test_data]


def sent2features(sent):
    return [feature18(sent, i) for i in range(len(sent))]

train_sents18 = [sent2features(s) for s in train_data]
val_sents18 = [sent2features(s) for s in val_data]
test_sents18 = [sent2features(s) for s in test_data]

In [ ]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)




# fitting the model for grid search 
crf.fit(train_sents1, train_labels)

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [ ]:
training_sent_list = [train_sents1,train_sents2,train_sents3,train_sents4,train_sents5,train_sents6,train_sents7,train_sents8,train_sents9,train_sents10,train_sents11,train_sents12,train_sents13,train_sents14,train_sents15,train_sents16,train_sents17,train_sents18]

In [ ]:
valid_sent_list = [val_sents1,val_sents2,val_sents3,val_sents4,val_sents5,val_sents6,val_sents7,val_sents8,val_sents9,val_sents10,val_sents11,val_sents12,val_sents13,val_sents14,val_sents15,val_sents16,val_sents17,val_sents18]

In [ ]:
crf.fit(train_sents4,train_labels)

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [ ]:
# display the result table
dfFeatures = {'features':['Stem','+Pos','+Chunk','+BOS','+EOS','+isUpper','+shape','+shortShape','+number','+hypen','+upper_digit and dash','+prefixes','+suffixes','+allUpper','+Stopword','+Neighbords','+sShape Neighbor','+wShape Neighbor','+inGazetter'],
              'F1_Score':[],
              'Accuracy':[],
              'Precision': []
              
              }
for i in range(len(training_sent_list)):
  crf.fit(training_sent_list[i],train_labels)


  y_pred = crf.predict(valid_sent_list[i])
  f1_score=metrics.flat_f1_score(val_labels, y_pred,
                      average='weighted', labels=labels)

  accuracy =metrics.flat_accuracy_score(val_labels, y_pred)
  precision =metrics.flat_precision_score(val_labels,y_pred,average = "weighted")

  
  dfFeatures['F1_Score'].append(f1_score)
  dfFeatures['Accuracy'].append(accuracy)
  dfFeatures['Precision'].append(precision)




In [ ]:
crf.fit(train_sents,train_labels)

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [ ]:
y_pred = crf.predict(val_sents)
f1_score=metrics.flat_f1_score(val_labels, y_pred,
                      average='weighted', labels=labels)
accuracy =metrics.flat_accuracy_score(val_labels, y_pred)
precision =metrics.flat_precision_score(val_labels,y_pred,average = "weighted")

print(f1_score,' ',accuracy,' ',precision)
dfFeatures['F1_Score'].append(f1_score)
dfFeatures['Accuracy'].append(accuracy)
dfFeatures['Precision'].append(precision)


0.978895261060282   0.9791190042266082   0.9788085755191352


In [ ]:
dfFeatures = pd.DataFrame(dfFeatures)
dfFeatures

,features,F1_Score,Accuracy,Precision
0,Stem,0.935849,0.939083,0.936609
1,+Pos,0.961719,0.962465,0.962056
2,+Chunk,0.962823,0.963337,0.963148
3,+BOS,0.962680,0.963182,0.963015
4,+EOS,0.962763,0.963240,0.963128
5,+isUpper,0.966670,0.966497,0.967505
6,+shape,0.970604,0.970802,0.970587
7,+shortShape,0.970775,0.970976,0.970807
8,+number,0.970910,0.971054,0.970943
9,+hypen,0.971161,0.971344,0.971181


We can see that from the table best model is 18 which is +inGazetter.
And now we will check classficiation report with test set.

In [ ]:
# display the classification report for the best model
crf.fit(train_sents,train_labels)

y_pred = crf.predict(test_sents)

sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    test_labels, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

           O      0.991     0.988     0.989     38554
       B-LOC      0.858     0.885     0.871      1668
       I-LOC      0.773     0.755     0.764       257
      B-MISC      0.808     0.789     0.798       702
      I-MISC      0.576     0.685     0.626       216
       B-ORG      0.822     0.760     0.790      1661
       I-ORG      0.707     0.758     0.732       835
       B-PER      0.855     0.863     0.859      1617
       I-PER      0.880     0.962     0.919      1156

    accuracy                          0.961     46666
   macro avg      0.808     0.827     0.817     46666
weighted avg      0.962     0.961     0.961     46666



## Recurrent Neural Network (RNN)

In [ ]:
import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Model, Input, Sequential
from keras.layers import Dense, Flatten, Embedding, Input, Dropout, LSTM, TimeDistributed, Bidirectional
from keras.callbacks import ModelCheckpoint

from gensim.models import Word2Vec
import gensim.downloader as api

In [ ]:
!pip install seqeval
from seqeval.metrics import f1_score, precision_score, recall_score, classification_report

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from collections import defaultdict

In [ ]:


train_words = [[s[0] for s in w] for w in train_data ]
val_words = [[s[0] for s in w] for w in val_data ]
test_words = [[s[0] for s in w] for w in test_data ]
labels_ = [[s[3] for s in w] for w in train_data ]
labels_for_test =[[s[3] for s in w] for w in test_data ] 

In [ ]:
sorted_labels.append('Others')
numlist = [0,1,2,3,4,5,6,7,8,9]

labels_dict = dict(zip(numlist,sorted_labels))

In [ ]:
labels_dict

{0: 'O',
 1: 'B-LOC',
 2: 'I-LOC',
 3: 'B-MISC',
 4: 'I-MISC',
 5: 'B-ORG',
 6: 'I-ORG',
 7: 'B-PER',
 8: 'I-PER',
 9: 'Others'}

In [ ]:
# Create your own word embeddings from scratch and load a pretrained word embeddings
corpus = api.load('text8')
                                            

pre_embedding = Word2Vec(corpus,window = 5,min_count = 1,workers = 4)
# You can check https://radimrehurek.com/gensim/models/word2vec.html for training a word embeddings from scratch

# You can check https://radimrehurek.com/gensim/auto_examples/howtos/run_downloader_api.html and https://github.com/RaRe-Technologies/gensim-data for loading pretrained word embeddings. 


[==================================================] 100.0% 31.6/31.6MB downloaded


In [ ]:
embedding_s = Word2Vec(sentences=train_words, window=5, min_count=1, workers=4)

In [ ]:
tokenizer = Tokenizer()
tokenizer_label = Tokenizer()

tokenizer.fit_on_texts(train_words)
tokenizer_label.fit_on_texts(labels_)



word_index = tokenizer.word_index # Tuple that contains words and their corresponding number
word_index_label  =tokenizer_label.word_index

In [ ]:
word_index_label

{'b-loc': 2,
 'b-misc': 7,
 'b-org': 4,
 'b-per': 3,
 'i-loc': 8,
 'i-misc': 9,
 'i-org': 6,
 'i-per': 5,
 'o': 1}

In [ ]:
num_of_words = len(list(tokenizer.word_index))
print("Number of word from our training word is:",num_of_words)

Number of word from our training word is: 21010


In [ ]:
train_sequence = tokenizer.texts_to_sequences(train_words)
val_sequence = tokenizer.texts_to_sequences(val_words)
test_sequence = tokenizer.texts_to_sequences(test_words)


label_sequence = tokenizer_label.texts_to_sequences(labels_)
# Return the list of sentences that contains their cooresding values, word--> sentences which labelled in word_index
tokenizer_label.fit_on_texts(labels_for_test)
test_label_sequence = tokenizer_label.texts_to_sequences(labels_for_test)

In [ ]:
sorted_labels

['O', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER']

In [ ]:
# Identify max length of token in our sequences 
max_length = 0
for index in range(len(train_sequence)):
  numberofwords=len(train_sequence[index])
  if (numberofwords) > (max_length):
    max_length = numberofwords
print(max_length)

113


In [ ]:
# preprare your dataset for RNN classifier (you need to add padding to labels as well)

train_pad = pad_sequences(train_sequence, maxlen=max_length,padding='post')
valid_pad = pad_sequences(val_sequence, maxlen=max_length,padding='post') 
test_pad = pad_sequences(test_sequence, maxlen=max_length,padding='post')  

label_pad = pad_sequences(label_sequence, maxlen=max_length,padding='post')

test_label_pad = pad_sequences(test_label_sequence,maxlen=max_length,padding='post')








### Embedding matrix  from our train data

In [ ]:
# Create Embedding Matrices and Layers

unique_words = len(word_index)
total_words = unique_words + 1
skipped_words = 0
embedding_dim = 100  
embedding_matrix = np.zeros((total_words, embedding_dim))
for word, index in tokenizer.word_index.items():
  try:
    embedding_vector = embedding_s[word]
  except:
    skipped_words = skipped_words+1
    pass
  if embedding_vector is not None:
    embedding_matrix[index] = embedding_vector
print("Embeddings Matrix shape of trained data: ",embedding_matrix.shape)


embedding_layer = Embedding(total_words, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False)









Embeddings Matrix shape of trained data:  (21011, 100)


In [ ]:
n_tags = len(sorted_labels)

### Models trained from embedding layer as training set

In [ ]:
# Create your models and train them

model = Sequential()
model.add(embedding_layer)

model.add(Bidirectional(LSTM(units=embedding_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode = 'concat'))

model.add(LSTM(units=embedding_dim, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))

model.add(TimeDistributed(Dense(n_tags+1, activation="relu")))


model.compile(optimizer='adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 113, 100)          2101100   
                                                                 
 bidirectional_10 (Bidirecti  (None, 113, 200)         160800    
 onal)                                                           
                                                                 
 lstm_19 (LSTM)              (None, 113, 100)          120400    
                                                                 
 time_distributed_6 (TimeDis  (None, 113, 10)          1010      
 tributed)                                                       
                                                                 
Total params: 2,383,310
Trainable params: 282,210
Non-trainable params: 2,101,100
_________________________________________________________________


In [ ]:
model.fit(train_pad, label_pad,epochs=5,batch_size=1000,verbose = 1)

Epoch 1/5
15/15 [==============================] - 153s 10s/step - loss: 0.0827 - accuracy: 0.9794
Epoch 2/5
15/15 [==============================] - 150s 10s/step - loss: 0.0821 - accuracy: 0.9795
Epoch 3/5
15/15 [==============================] - 148s 10s/step - loss: 0.0814 - accuracy: 0.9796
Epoch 4/5
15/15 [==============================] - 149s 10s/step - loss: 0.0808 - accuracy: 0.9798
Epoch 5/5
15/15 [==============================] - 149s 10s/step - loss: 0.0803 - accuracy: 0.9799


In [ ]:
scores = model.evaluate(test_pad, test_label_pad, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 97.03%


In [ ]:
test_label_pad

array([[1, 0, 0, ..., 0, 0, 0],
       [4, 1, 7, ..., 0, 0, 0],
       [3, 5, 0, ..., 0, 0, 0],
       ...,
       [4, 1, 4, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [4, 1, 4, ..., 0, 0, 0]], dtype=int32)

In [ ]:
y_pred = model.predict(test_pad)

labels = np.argmax(y_pred, axis=1) 

In [ ]:
# define a function to remove paddings and align labels and tokens
def align_predictions(predictions:np.array, label_ids:np.array) -> list, list:
  pass





  

In [ ]:
# Evaluate your models with functions of seqeval library









In [ ]:
# ....

## My Report
We have two different models which first one if Conditional Random Fields(CRF) and Reccurent Neural Network(RNN), we use this models to NER.

# **CRF Model:**

In this model first we create a gazetter from the 20,000 wikipedia pages. Then use this gazeeter to add feature in crf model. We trained 19 different models for crf which we add one by one new feature to each model  in order to see how well our model is learning for each feature.
At the end of the all traning models we set that the last one which is checks all the features that lastly add "+inGazetter" feauture and evaluate in the test data.








In [ ]:
dfFeatures

,features,F1_Score,Accuracy,Precision
0,Stem,0.935849,0.939083,0.936609
1,+Pos,0.961719,0.962465,0.962056
2,+Chunk,0.962823,0.963337,0.963148
3,+BOS,0.962680,0.963182,0.963015
4,+EOS,0.962763,0.963240,0.963128
5,+isUpper,0.966670,0.966497,0.967505
6,+shape,0.970604,0.970802,0.970587
7,+shortShape,0.970775,0.970976,0.970807
8,+number,0.970910,0.971054,0.970943
9,+hypen,0.971161,0.971344,0.971181


When we check the F1, Accuracy and Precision score for all models, firstly we began with only stemming of word which gives scores as:

               F1         Accuracy  Precision

      Stem	0.935849	0.939083	0.936609

  
This scores are good values to start, and we can see that mostly our scores are goes up for new features or stays similar to last ones.

For instance: 

  
    features	F1_Score	Accuracy	Precision
    0	Stem	0.935849	0.939083	0.936609
    1	+Pos	0.961719	0.962465	0.962056
    2	+Chunk	0.962823	0.963337	0.963148
    3	+BOS	0.962680	0.963182	0.963015


At the end of the +BOS our f1, accuracy and precision scores are decreased a little to 0.0002 which is not affect the model hugely.

We have two main points that features affect positively to our model which first one is:

        features	F1_Score	Accuracy	Precision
    5	+isUpper	0.966670	0.966497	0.967505
    6	+shape	0.970604	0.970802	0.970587


we can see that after we add +sShape our model score are changes by 0.004 which normally we can not see that much increase.

and secondly


        features	F1_Score	Accuracy	Precision
    14	+Stopword	0.972693	0.972818	0.972659
    15	+Neighbords	0.976711	0.976967	0.976607

again there is score increase by 0.0041 by adding neighbord words of that given words.

At the end we can see our last model is the best one and from the beggining with the **0.935** F1 score is up to **0.978** with the help of the 19 different features.

  After this step we trained best model from above with the test data and report as:

                  precision    recall  f1-score   support

           O      0.991     0.988     0.989     38554
       B-LOC      0.858     0.885     0.871      1668
       I-LOC      0.773     0.755     0.764       257
      B-MISC      0.808     0.789     0.798       702
      I-MISC      0.576     0.685     0.626       216
       B-ORG      0.822     0.760     0.790      1661
       I-ORG      0.707     0.758     0.732       835
       B-PER      0.855     0.863     0.859      1617
       I-PER      0.880     0.962     0.919      1156

    accuracy                          0.961     46666
    macro avg      0.808     0.827     0.817     46666
    weighted avg      0.962     0.961     0.961     46666


We can now check each scores for each tags in test data. We have 4 scores as 0,I-PER,B-LOCK and B-PER. Since we have lots of 0 which we can also see from the support. We expect to label 0 with this close accuracy.

We have worst scores as I-MISC, I-ORG,I-LOC which f1 scores as 0.62,0.73, 0.76.
The interesting part is even if the I-MISC AND I-LOC has similar support values, their their precision and f1 values are differ from each other.

Our model is not good as label I-MISC but we can not say that this is because of our data did not contains I-MISC tags since our  I-LOC values are better than I-MISC.

RNN MODEL:

For rnn, we again do similar things as CNN, word embeding and layers are changes. I can only do one rnn model to fit with the word embedding strategy as from our training data. Rnn is differ from cnn with the Bidirectional and Time Distributed layer
and accuracy of this model is

 Accuracy: 97.03%

###Refferences

https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html 


https://colab.research.google.com/drive/1FO48RJgao6_w8cICPKNsveisKz-OFsPP#scrollTo=oCT-Zt3ZMocG (From the first project )